# Baseline Full Integration Model - Final Model Building and Training

Import Julia machine learning packages, plotting package, and file loader

In [1]:
using Flux,Statistics,Plots,MLDataUtils,DelimitedFiles
using Flux.Data: DataLoader
include("utilityfunc.jl");

Set the upper and lower bounds on the features and define the scaling and unscaling functions

In [2]:
#set lower and upper bounds on Lpt, Kt, Rs,(Lpt is first element, Kt is second element, Rs is the third element)
lb=Float32[5e-7,5e-7,5]
ub=Float32[5e-6,5e-6,30]

3-element Vector{Float32}:
  5.0f-6
  5.0f-6
 30.0

In [3]:
#Define scaler functions to preprocess data
conc_ub=Float32(.6)

iscaler(x)=(x-lb)./(ub-lb) #Min-Max Normalization on Features
inv_iscaler(x)=x.*(ub.-lb).+lb
oscaler(x)= x ./ conc_ub#Min-Max Normalization on Targets
inv_oscaler(x)=x.*conc_ub

inv_oscaler (generic function with 1 method)

In [4]:
function iscalerbatch(x)
    nfeat,sl = size(x)
    out=Array{Float32,2}(undef,nfeat,sl)
    for i=1:sl
        out[:,i]=iscaler(x[:,i])
    end
    return out
end

iscalerbatch (generic function with 1 method)

In [5]:
function inv_iscalerbatch(x)
    nfeat,sl = size(x)
    out=Array{Float32,2}(undef,nfeat,sl)
    for i=1:sl
        out[:,i]=inv_iscaler(x[:,i])
    end
    return out
end

inv_iscalerbatch (generic function with 1 method)

Define model characteristics, load the data, reshape the data into proper form, and scale the data

In [6]:
n_samp=Int64(1e4); # number of samples
nbatch = n_samp; # number of batches
nfeat=3; #5 features
ntarg=20; #1 target values

In [7]:
d = readdlm("MLFinalData_v3.csv",',',Float32)
X_p= d[1:5,:];
Y_p= d[6,:];
X=Array{Float32,2}(undef,3,n_samp)
for i=1:n_samp
    X[:,i]=X_p[1:3,20*i]
end
Y=reshape(Y_p,(20,n_samp));

In [8]:
X=iscalerbatch(X);

In [9]:
Y = Float32.(oscaler(Y));

Shuffle the data, split the data into training, validation, and test sets using 80/10/10 Split

In [10]:
Xs,Ys=shuffleobs((X, Y));
(x_train, y_train), (x_val, y_val),(x_test, y_test) = splitobs((Xs, Ys), at = (0.8,.10) );
data = DataLoader((x_train,y_train));

Define model structure- here we use 2 hidden layers with 8 nodes and the ReLU activation function and the MSE loss function

In [16]:
n_hidden=8;
activation_func=Flux.relu
function loss(x, y)
     Flux.mse(m_baseline(x), y)
end
m_baseline = Chain(Dense(nfeat,n_hidden),Dense(n_hidden,n_hidden,activation_func),Dense(n_hidden,n_hidden,activation_func),Dense(n_hidden,ntarg))
ps = params(m_baseline)
ϵ=2.5e-6; #Early stopping criteria
opt= ADAM(5e-4);

Train for 100 epochs using the ADAM optimizer with an exponentially decaying learning rate

In [ ]:
n_epochs = 100;
e_log = zeros(n_epochs)
decay(epoch)=1e-3*exp(-.023*epoch)
for i=1:100
    Flux.train!(loss, ps, data, ADAM(decay(i)))
    e_log[i]=loss(x_val,y_val)
    if (i%1 == 0 || i ==1)
        @show(i,e_log[i])
    end
    if (e_log[i]<ϵ)
        break
    end
end

Evaluate performance metrics on the test set

In [18]:
@show loss(x_test,y_test)
@show sqrt(loss(x_test,y_test))
@show mean_error_func(x_test,y_test,m_baseline)

loss(x_test, y_test) = 6.2188738f-6
sqrt(loss(x_test, y_test)) = 0.002493767f0
mean_error_func(x_test, y_test, m_baseline) = 0.9576018f0


0.9576018f0

In [14]:
#Export the model (uncomment code)

In [15]:
using BSON: @save
#@save "m_baseline.bson" m_baseline